following: https://colab.research.google.com/drive/1ci4oCqKmYMBJdDCH8FsZx6lvaORYfh7u?usp=sharing#scrollTo=vXa06Afvd76e

In [1]:
%%capture
!pip install --upgrade plotly
!pip install -U kaleido

In [2]:
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/MyDrive/Courses/Fall 2021/dlsys/bnn-cf-vs-robust"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Courses/Fall 2021/dlsys/bnn-cf-vs-robust


In [3]:
import os, glob 
import pandas as pd 
import numpy as np 
import yaml 
from pathlib import Path
from tqdm.notebook import tqdm 


In [4]:
data_root = Path('data/output/exp1-pmnist-robustness')
fig_root = Path('figures/exp1-pmnist_robustness')

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import numpy as np

In [6]:
# FGSM attack code
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image


def test( model, device, test_loader, epsilon , calc_example = False):

    # Accuracy counter
    correct = 0
    adv_examples = []

    # Loop over all examples in test set
    for data, target in test_loader:

        # Send the data and label to the device
        data, target = data.to(device), target.to(device)

        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # Forward pass the data through the model
        output = model(data)
        init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

        # If the initial prediction is wrong, dont bother attacking, just move on
        if init_pred.item() != target.item():
            continue

        # Calculate the loss
        loss = F.nll_loss(output, target)

        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()

        # Collect datagrad
        data_grad = data.grad.data

        # Call FGSM Attack
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        # Re-classify the perturbed image
        output = model(perturbed_data)

        # Check for success
        final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        if final_pred.item() == target.item():
            correct += 1
            # Special case for saving 0 epsilon examples
            if (epsilon == 0) and (len(adv_examples) < 5) and calc_example:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        else:
            # Save some adv examples for visualization later
            if len(adv_examples) < 5 and calc_example:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )

    # Calculate final accuracy for this epsilon
    final_acc = correct/float(len(test_loader))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(test_loader), final_acc))

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples

In [7]:
from src.models_utils import BNN
from src.pmnist_robustness_data_utils import TaskDataSet

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.0,), std=(1.0,))
])

common_dload_args = dict(
    batch_size  = 1
)

test_dataset = TaskDataSet('data/input/pmnist_robustness/task-01/original/test', transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=True, **common_dload_args)

In [8]:
def test_fgsm(model_state_path, epsilons): 
    ckpt = torch.load(model_state_path)
    model = BNN(**ckpt['model_args'])
    model.load_state_dict(ckpt['model_states'])
    model.eval()
    model.cuda()

    accuracies = []

    for eps in tqdm(epsilons, desc='epsi'):
        acc, _ = test(model, 'cuda:0', test_loader, eps)
        accuracies.append(acc)
    return accuracies

data_dirs = os.listdir(data_root) 

epsilons = [0, .05, .1, .15, .2]

dfs = []

for data_dir in tqdm(data_dirs, desc='dir'):
    exp_config = yaml.safe_load(open(data_root / data_dir / 'exp-config.yaml', 'r'))
    meta_val = exp_config['meta'][0]

    model_path = data_root / data_dir / 'models'
    model_state_files = os.listdir(model_path)
    model_state_files = list(np.sort(model_state_files))

    # get only beginning and end for now
    model_state_files = [model_state_files[0], model_state_files[-1]]

    for train_phase in tqdm(model_state_files, desc='train phase'):
        fgsm_acc = test_fgsm(model_path / train_phase, epsilons)
        
        dfs.append(pd.DataFrame(dict(
            meta = meta_val,
            train_phase = train_phase,
            epsilons = epsilons,
            fgsm_acc = fgsm_acc
        )))

dir:   0%|          | 0/3 [00:00<?, ?it/s]

train phase:   0%|          | 0/2 [00:00<?, ?it/s]

epsi:   0%|          | 0/5 [00:00<?, ?it/s]

Epsilon: 0	Test Accuracy = 9747 / 10000 = 0.9747
Epsilon: 0.05	Test Accuracy = 7399 / 10000 = 0.7399
Epsilon: 0.1	Test Accuracy = 2345 / 10000 = 0.2345
Epsilon: 0.15	Test Accuracy = 503 / 10000 = 0.0503
Epsilon: 0.2	Test Accuracy = 111 / 10000 = 0.0111


epsi:   0%|          | 0/5 [00:00<?, ?it/s]

...

train phase:   0%|          | 0/2 [00:00<?, ?it/s]

epsi:   0%|          | 0/5 [00:00<?, ?it/s]

Epsilon: 0	Test Accuracy = 9767 / 10000 = 0.9767
Epsilon: 0.05	Test Accuracy = 6972 / 10000 = 0.6972
Epsilon: 0.1	Test Accuracy = 2002 / 10000 = 0.2002
Epsilon: 0.15	Test Accuracy = 331 / 10000 = 0.0331
Epsilon: 0.2	Test Accuracy = 39 / 10000 = 0.0039


epsi:   0%|          | 0/5 [00:00<?, ?it/s]

Epsilon: 0	Test Accuracy = 1097 / 10000 = 0.1097
Epsilon: 0.05	Test Accuracy = 230 / 10000 = 0.023
Epsilon: 0.1	Test Accuracy = 23 / 10000 = 0.0023
Epsilon: 0.15	Test Accuracy = 2 / 10000 = 0.0002
Epsilon: 0.2	Test Accuracy = 0 / 10000 = 0.0


In [48]:
df = pd.concat(dfs, ignore_index=True)
df['train_phase'] = df['train_phase'].apply(lambda x: x.replace('.pt', ''))
df.fgsm_acc *= 100
df = df.sort_values(by=['epsilons','meta']).reset_index(drop=True)

In [13]:
import plotly.express as px
import plotly.graph_objects as go

In [14]:
axis_config = dict(
    showline=True,
    showgrid=False,
    showticklabels=True,
    linecolor='rgb(0, 0, 0)',
    linewidth=2,    
    ticks='outside',
    tickwidth=2
    )

font_config = dict(
    family="Fira Sans",
    size=15,
    color='black'
    )

title_config = dict(
    title_x = 0.5,
    title_y = 0.95,
    title_xanchor = 'center',
    title_yanchor = 'top',
    title_font_size=20
)

general_layout = go.Layout(
    xaxis=axis_config,
    yaxis=axis_config,
    font=font_config,
    margin=dict(autoexpand=True,l=100,r=50,t=100,b=120),
    showlegend=True,
    plot_bgcolor='white',
    autosize=True,
    **title_config
)

for i in range(1,5): # for subplots
    general_layout['xaxis' + str(i)] = axis_config
    general_layout['yaxis' + str(i)] = axis_config

In [64]:
fig = px.line(
    df, 
    x="epsilons", 
    y="fgsm_acc", 
    color="meta",
    symbol="train_phase",
    color_discrete_sequence=px.colors.sequential.Burg,
    markers=True
)


fig.update_traces(line=dict(width=4), marker_size=20)

fig.update_layout(
    general_layout,
    height=600, width=880,    
    title_y = 0.92, 
    font_size = 25, title_font_size = 30,
    title_text = 'Meta-train BNN model in response to FGSM attack',
    legend=dict(
        yanchor="top",
        y=1,
        xanchor="right",
        x=0.99
    )
)


fig.write_image(fig_root / 'vary-meta-fgsm-test.svg')

fig.show()